In [80]:
import numpy as np
import re
import json
import wikipedia
from string import punctuation
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from collections import Counter
import textdistance
punct = set(punctuation)

In [37]:
corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [38]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [39]:
def get_closest_match_vec(text, X, vec, TOPN=5):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:TOPN]
    
    return [id2word[top] for top in topn]

In [40]:
def get_closest_match_with_metric(text, X, vec, metric=textdistance.levenshtein):
    similarities = Counter()
    lookup = get_closest_match_vec(text, X, vec, TOPN=3)
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(1)[0]

In [41]:
get_closest_match_with_metric('опофиоз', X, vec, textdistance.hamming)

('апофеоз', 0.7142857142857143)

Мы создали функцию, которая сначала находит пять ближайших соответствий по символьному векторному представлению слов, а затем, для этих пяти считает расстояние Левенштейна и выделяет наилучший вариант. Теперь проверим качество её работы на материалах Диалога.

In [42]:
corpus = open('corpus_ng.txt', encoding='utf8').read().splitlines()
vocab = set()
for line in corpus:
    for word in line.split():       
        vocab.add(word.lower())
# Лишний???

In [43]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [45]:
def align_words(sent_1, sent_2):
    punct = set(punctuation)
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [55]:
mistakes = []
total = 0

for i in range(len(true)):
    
    word_pairs = align_words(true[i], bad[i])
    
    for pair in word_pairs:
        if pair[0] != pair[1]:
            mistakes.append(pair)
            total += 1

In [56]:
correct = 0
failed = []
for pair in mistakes:
    correction = get_closest_match_with_metric(pair[1], X, vec, textdistance.hamming)[0]
    if correction == pair[0]:
        correct += 1
    else:
        failed.append(pair[1])
print(correct)

391


In [53]:
print(correct/total)

0.2996168582375479


т. е. наша функция справилась только с 30 % ошибок из корпуса. На первый взгляд, это значение может показаться не слишком большим. Однако если вспомнить, что мы работаем на материалах газетного корпуса, а лексика СМИ всё же достаточно ограничена, это не так уж и плохо.Посмотрим на ошибки 

In [121]:
print(failed[:50])

['симпатичнейшое', 'пояним', 'полчатся', 'нащщот', 'основая', 'вобщем', 'как', 'вы', 'знаете', 'из', 'моего', 'не', 'давнего', 'ящека', 'хороше', 'патаму', 'шта', 'поффтыкав', 'соре', 'чтото', 'мошный', 'хороше', 'седня', 'вешать', 'эт', 'канешна', 'начальнег', 'павзрослому', 'подсаживаеться', 'какието', 'малолетнии', 'монголойдом', 'баръер', 'коментариев', 'отвественный', 'расчитаны', 'обълись', 'распрашивая', 'зубодробительня', 'каааак', 'расщифровать', 'самойто', 'тул', 'ваще', 'тока', 'навернео', 'ооочень', 'както', 'мущщину', 'никада']


Часть из этих ошибок (гуливера, афальтоукладчик, начальнег) неисправимы на нашем корпусе (да и на любом маленьком корпусе), часть (любви, женщины) -, по-видимому, ошибки в выборе падежа и тоже нами не покроются, но некоторые вещи можно исправить увеличением N-грамм (ооооочень)

Программа ниже отличается заменой  TfidfVectorizer(analyzer='char', ngram_range=(1,1)) на vec = CountVectorizer(analyzer='char', ngram_range=(1,3))

In [58]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = CountVectorizer(analyzer='char', ngram_range=(1,3))
X = vec.fit_transform(vocab)

In [59]:
correct = 0
failed = []
for pair in mistakes:
    correction = get_closest_match_with_metric(pair[1], X, vec, textdistance.hamming)[0]
    if correction == pair[0]:
        correct += 1
    else:
        failed.append(pair[1])
print(correct)

520


In [60]:
print(correct/total)

0.39846743295019155


Ура! мы добились увелечения точности на 10 %. Теперь добавим новый корпус. Так у нас есть небольшой корпус из википедии.

In [ ]:
wikipedia.set_lang('ru')
wiki_content = []
pages = wikipedia.random(500)
    
for page_name in pages:
    try:
        page = wikipedia.page(page_name)
        
    except Exception as e:
        print('Skipping page {}'.format(page_name), e)
        continue
    wiki_content.append('{}\n{}'.format(page.title, page.content.replace('==', '')))
for sent in wiki_content.split():
    WORDS.update(sent)
    
for sent in wiki_content.split():
    WORDS.update(sent)

# слишком сложно

Я хотел скачать 500 статей с Вики, но, видимо, моему компу это слишком сложно, так что воспользуемся имеющейся сотней и докинем её в наш корпус

In [116]:
wiki_texts = json.loads(open('wiki_texts.json', 'r', encoding='utf8').read())


In [89]:
table = str.maketrans({ch: None for ch in punctuation})
for sent in wiki_texts['ru']:
    for word in  sent.split():
        word = word.translate(table)
        WORDS.update(word.lower())

In [90]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = CountVectorizer(analyzer='char', ngram_range=(1,3))
X = vec.fit_transform(vocab)

In [91]:
correct = 0
failed = []
for pair in mistakes:
    correction = get_closest_match_with_metric(pair[1], X, vec, textdistance.hamming)[0]
    if correction == pair[0]:
        correct += 1
    else:
        failed.append(pair[1])
print(correct)

519


Как ни странно корпус из википедии на этих данных нам нисколько не помог. Результат почти тот же. Может, просто он недостаточно большой?

Теперь возьмём материалы "Диалога" и, исходя из предположения о том, что люди ошибаются не в каждом слове, но спеллчекер может пометить верные слова как неверные, прогоним его через наш алгоритм (добавив еще один уровень, определяющий верность-неверность слова). Будем считать, что материалы "Диалога" - симмуляция того, как часто люди ошибаются. Какой процент верного текста мы получим?

In [115]:
percent = 0
all = 0
fail = []
def final(word, vocab, X, vec):
         if word in vocab:
            return word
         else:
            return get_closest_match_with_metric(word, X, vec, textdistance.hamming)[0]

for layer1, layer2 in zip(true, bad):
    for item in align_words(layer1, layer2):
        if item[0] == final(item[1], vocab, X, vec):
            percent += 1
        else:
            fail.append((item[0], item[1], final(item[1], vocab, X, vec)))
        all +=1

In [112]:
print(percent)

8282


In [113]:
print(percent/all)

0.8272073511785857


То есть при нашей помощи только 83 % текста в итоге будут верными. Посмотрим на то, что не удалось исправить. Во многом это вещи связанные с, по-видимому, отсутсвием более редких форм типа "симпатичнейшее" или "подсаживается" в словаре и неправильным выбором  части речи ("хороше" стало "хорошее", а не "хорошо" (с точки зрения парсера оба варианта почти однаково логичны)). Эти проблемы могли бы быть решены подключением синтаксических парсеров, а также, возможно, частично майстемом, но этим мы сейчас заниматься не будем.

In [119]:
print(fail[:50])


[('симпатичнейшее', 'симпатичнейшое', 'симпатичный'), ('шпионское', 'шпионское', 'шпионской'), ('гламурный', 'гламурный', 'гламур'), ('бонда', 'бонда', 'фонда'), ('superheadz', 'superheadz', 'super'), ('clap', 'clap', 'ap'), ('camera', 'camera', 'america'), ('поясним', 'пояним', 'помним'), ('получатся', 'полчатся', 'полчаса'), ('язычки', 'язычки', 'язычка'), ('милые', 'милые', 'милы'), ('насчет', 'нащщот', 'нащокина'), ('чавеса', 'чавеса', 'чавес'), ('основная', 'основая', 'основа'), ('попавшим', 'попавшим', 'попавших'), ('аварийно-спасательных', 'аварийно-спасательных', 'аварийно-восстановительных'), ('в', 'вобщем', 'всеобщем'), ('общем', 'как', 'как'), ('как', 'вы', 'вы'), ('вы', 'знаете', 'знаете'), ('знаете', 'из', 'из'), ('из', 'моего', 'моего'), ('моего', 'не', 'не'), ('недавнего', 'давнего', 'давнего'), ('пропажу', 'пропажу', 'пропал'), ('ящика', 'ящека', 'щеках'), ('почте.ру', 'почте.ру', 'почте'), ('хорошо', 'хороше', 'хорошее'), ('рите', 'рите', 'жрите'), ('потому', 'патаму',